# import libraries

In [1]:
!pip install parsivar


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import json
from parsivar import FindStems
from collections import defaultdict
import math
import re
from re import sub
import time
import sys

# Reading Dataset

In [3]:
json_file_path = 'IR_data_news_12k.json'

with open(json_file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

# Extract content and use doc index as doc_id
documents = [data[i]['content'] for i in data]
urls = [data[i]['url'] for i in data]
titles = [data[i]['title'] for i in data]

# Preprocessing data

## Difining normalizer

In [4]:
class MyNormalizer:

    def __init__(self):
        with open('verbs.json', 'r') as file:
            self.verbs = json.load(file)
        with open('words.json', 'r') as file:
            self.words = json.load(file)


    def normalize_alphabets(self,doc_string):
        a0 = "ء"
        b0 = "ئ"
        c0 = sub(a0, b0, doc_string)
        a1 = r"ٲ|ٱ|إ|ﺍ|أ"
        a11 = r"ﺁ|آ"
        b1 = r"ا"
        b11 = r"آ"
        c11 = sub(a11, b11, c0)
        c1 = sub(a1, b1, c11)
        a2 = r"ﺐ|ﺏ|ﺑ"
        b2 = r"ب"
        c2 = sub(a2, b2, c1)
        a3 = r"ﭖ|ﭗ|ﭙ|ﺒ|ﭘ"
        b3 = r"پ"
        c3 = sub(a3, b3, c2)
        a4 = r"ﭡ|ٺ|ٹ|ﭞ|ٿ|ټ|ﺕ|ﺗ|ﺖ|ﺘ"
        b4 = r"ت"
        c4 = sub(a4, b4, c3)
        a5 = r"ﺙ|ﺛ"
        b5 = r"ث"
        c5 = sub(a5, b5, c4)
        a6 = r"ﺝ|ڃ|ﺠ|ﺟ"
        b6 = r"ج"
        c6 = sub(a6, b6, c5)
        a7 = r"ڃ|ﭽ|ﭼ"
        b7 = r"چ"
        c7 = sub(a7, b7, c6)
        a8 = r"ﺢ|ﺤ|څ|ځ|ﺣ"
        b8 = r"ح"
        c8 = sub(a8, b8, c7)
        a9 = r"ﺥ|ﺦ|ﺨ|ﺧ"
        b9 = r"خ"
        c9 = sub(a9, b9, c8)
        a10 = r"ڏ|ډ|ﺪ|ﺩ"
        b10 = r"د"
        c10 = sub(a10, b10, c9)
        a11 = r"ﺫ|ﺬ|ﻧ"
        b11 = r"ذ"
        c11 = sub(a11, b11, c10)
        a12 = r"ڙ|ڗ|ڒ|ڑ|ڕ|ﺭ|ﺮ"
        b12 = r"ر"
        c12 = sub(a12, b12, c11)
        a13 = r"ﺰ|ﺯ"
        b13 = r"ز"
        c13 = sub(a13, b13, c12)
        a14 = r"ﮊ"
        b14 = r"ژ"
        c14 = sub(a14, b14, c13)
        a15 = r"ݭ|ݜ|ﺱ|ﺲ|ښ|ﺴ|ﺳ"
        b15 = r"س"
        c15 = sub(a15, b15, c14)
        a16 = r"ﺵ|ﺶ|ﺸ|ﺷ"
        b16 = r"ش"
        c16 = sub(a16, b16, c15)
        a17 = r"ﺺ|ﺼ|ﺻ"
        b17 = r"ص"
        c17 = sub(a17, b17, c16)
        a18 = r"ﺽ|ﺾ|ﺿ|ﻀ"
        b18 = r"ض"
        c18 = sub(a18, b18, c17)
        a19 = r"ﻁ|ﻂ|ﻃ|ﻄ"
        b19 = r"ط"
        c19 = sub(a19, b19, c18)
        a20 = r"ﻆ|ﻇ|ﻈ"
        b20 = r"ظ"
        c20 = sub(a20, b20, c19)
        a21 = r"ڠ|ﻉ|ﻊ|ﻋ"
        b21 = r"ع"
        c21 = sub(a21, b21, c20)
        a22 = r"ﻎ|ۼ|ﻍ|ﻐ|ﻏ"
        b22 = r"غ"
        c22 = sub(a22, b22, c21)
        a23 = r"ﻒ|ﻑ|ﻔ|ﻓ"
        b23 = r"ف"
        c23 = sub(a23, b23, c22)
        a24 = r"ﻕ|ڤ|ﻖ|ﻗ"
        b24 = r"ق"
        c24 = sub(a24, b24, c23)
        a25 = r"ڭ|ﻚ|ﮎ|ﻜ|ﮏ|ګ|ﻛ|ﮑ|ﮐ|ڪ|ك"
        b25 = r"ک"
        c25 = sub(a25, b25, c24)
        a26 = r"ﮚ|ﮒ|ﮓ|ﮕ|ﮔ"
        b26 = r"گ"
        c26 = sub(a26, b26, c25)
        a27 = r"ﻝ|ﻞ|ﻠ|ڵ"
        b27 = r"ل"
        c27 = sub(a27, b27, c26)
        a28 = r"ﻡ|ﻤ|ﻢ|ﻣ"
        b28 = r"م"
        c28 = sub(a28, b28, c27)
        a29 = r"ڼ|ﻦ|ﻥ|ﻨ"
        b29 = r"ن"
        c29 = sub(a29, b29, c28)
        a30 = r"ވ|ﯙ|ۈ|ۋ|ﺆ|ۊ|ۇ|ۏ|ۅ|ۉ|ﻭ|ﻮ|ؤ"
        b30 = r"و"
        c30 = sub(a30, b30, c29)
        a31 = r"ﺔ|ﻬ|ھ|ﻩ|ﻫ|ﻪ|ۀ|ە|ة|ہ"
        b31 = r"ه"
        c31 = sub(a31, b31, c30)
        a32 = r"ﭛ|ﻯ|ۍ|ﻰ|ﻱ|ﻲ|ں|ﻳ|ﻴ|ﯼ|ې|ﯽ|ﯾ|ﯿ|ێ|ے|ى|ي"
        b32 = r"ی"
        c32 = sub(a32, b32, c31)
        a33 = r'¬'
        b33 = r'‌'
        c33 = sub(a33, b33, c32)
        pa0 = r'•|·|●|·|・|∙|｡|ⴰ'
        pb0 = r'.'
        pc0 = sub(pa0, pb0, c33)
        pa1 = r',|٬|٫|‚|，'
        pb1 = r'،'
        pc1 = sub(pa1, pb1, pc0)
        pa2 = r'ʕ'
        pb2 = r'؟'
        pc2 = sub(pa2, pb2, pc1)
        na0 = r'0|٠'
        nb0 = r'۰'
        nc0 = sub(na0, nb0, pc2)
        na1 = r'1|١'
        nb1 = r'۱'
        nc1 = sub(na1, nb1, nc0)
        na2 = r'2|٢'
        nb2 = r'۲'
        nc2 = sub(na2, nb2, nc1)
        na3 = r'3|٣'
        nb3 = r'۳'
        nc3 = sub(na3, nb3, nc2)
        na4 = r'4|٤'
        nb4 = r'۴'
        nc4 = sub(na4, nb4, nc3)
        na5 = r'5'
        nb5 = r'۵'
        nc5 = sub(na5, nb5, nc4)
        na6 = r'۶|٦'
        nb6 = r'6'
        nc6 = sub(na6, nb6, nc5)
        na7 = r'7|٧'
        nb7 = r'۷'
        nc7 = sub(na7, nb7, nc6)
        na8 = r'8|٨'
        nb8 = r'۸'
        nc8 = sub(na8, nb8, nc7)
        na9 = r'9|٩'
        nb9 = r'۹'
        nc9 = sub(na9, nb9, nc8)
        ea1 = r'ـ|ِ|ُ|َ|ٍ|ٌ|ً|'
        eb1 = r''
        ec1 = sub(ea1, eb1, nc9)
        Sa1 = r'( )+'
        Sb1 = r' '
        Sc1 = sub(Sa1, Sb1, ec1)
        Sa2 = r'(\n)+'
        Sb2 = r'\n'
        Sc2 = sub(Sa2, Sb2, Sc1)
        return Sc2


    def normalize_unicodes(self,text):
        unicodes = [("﷽", "بسم الله الرحمن الرحیم"),
                    ("﷼", "ریال"),
                    ("(ﷰ|ﷹ)", "صلی"),
                    ("ﷲ", "الله"),
                    ("ﷳ", "اکبر"),
                    ("ﷴ", "محمد"),
                    ("ﷵ", "صلعم"),
                    ("ﷶ", "رسول"),
                    ("ﷷ", "علیه"),
                    ("ﷸ", "وسلم"),
                    ("ﻵ|ﻶ|ﻷ|ﻸ|ﻹ|ﻺ|ﻻ|ﻼ", "لا")]
        
        for old, new in unicodes:
            text = sub(old,new,text)
            return text


    def normalize_special_characters(self,text):

        removed_unnecessary_dots_text = re.sub(r'(?<!\d)\.|\.(?!\d)', '', text)

        cleaned_text = re.sub(r'[^\w\s\d@_.-]', '', removed_unnecessary_dots_text)

        return cleaned_text


    def simple_tokenizer(self,text):
        
        split_pattern = r'[ \t\u200c]+'

        tokens = re.split(split_pattern, text)

        tokens = [token for token in tokens if token]

        return tokens

    def fix_extra_spaces(self,text):

        extra_space_patterns = [
                  (r" {2,}", " "),  # remove extra spaces
                  (r"\n{3,}", "\n\n"),  # remove extra newlines
                  (r"\u200c{2,}", "\u200c"),  # remove extra ZWNJs
                  (r"\u200c{1,} ", " "),  # remove unneded ZWNJs before space
                  (r" \u200c{1,}", " "),  # remove unneded ZWNJs after space
                  (r"\b\u200c*\B", ""),  # remove unneded ZWNJs at the beginning of words
                  (r"\B\u200c*\b", ""),  # remove unneded ZWNJs at the end of words
                  (r"[ـ\r]", ""),  # remove keshide, carriage returns
              ]
        for old, new in extra_space_patterns:
            text = sub(old,new,text)

        return text

    def fix_number_spaces(self, text):

        number_space_patterns = [(r"(\d)([آابپتثجچحخدذرزژسشصضطظعغفقکگلمنوهی])", r"\1 \2"),
                  (r"([آابپتثجچحخدذرزژسشصضطظعغفقکگلمنوهی])(\d)", r"\1 \2"),
              ]
        for old, new in number_space_patterns:
            text = sub(old,new,text)

        return text
    def fix_affix_spaces(self,text):

        affix_spacing_patterns = [
                (r"([^ ]ه) ی ", r"\1‌ی "),  # fix ی space
                (r"(^| )(ن?می) ", r"\1\2‌"),  # put zwnj after می, نمی
                # put zwnj before تر, تری, ترین, گر, گری, ها, های
                (
                    r"(?<=[^\n\d "
                    + "]{2}) (تر(ین?)?|گری?|های?)(?=[ \n"
                    + "]|$)",
                    r"‌\1",
                ),
                # join ام, ایم, اش, اند, ای, اید, ات
                (
                    r"([^ ]ه) (ا(م|یم|ش|ند|ی|ید|ت))(?=[ \n" + "]|$)",
                    r"\1‌\2",
                ),
                # شنبهها => شنبه‌ها
                ("(ه)(ها)", r"\1‌\2"),
            ]
        
        for old, new in affix_spacing_patterns:
            text = sub(old,new,text)
        return text
    

    def correct_spacing(self,text):

        text = self.fix_extra_spaces(text)
        
        text = self.fix_number_spaces(text)

        lines = text.split("\n")
        result = []
        for line in lines:
            tokens = self.simple_tokenizer(line)
            spaced_tokens = self.token_spacing(tokens)
            line = " ".join(spaced_tokens)
            result.append(line)

        text = "\n".join(result)
        
        text = self.fix_affix_spaces(text)

        return text


    def token_spacing(self,tokens):

        suffixes = {
                "ی",
                "ای",
                "ها",
                "های",
                "هایی",
                "تر",
                "تری",
                "ترین",
                "گر",
                "گری",
                "ام",
                "ات",
                "اش",
            }
        
        result = []
        
        for t, token in enumerate(tokens):
            joined = False

            if result:
                token_pair = result[-1] + "‌" + token
                if (
                    token_pair in self.verbs
                    or token_pair in self.words
                    and self.words[token_pair][0] > 0
                ):
                    joined = True

                    if (
                        t < len(tokens) - 1
                        and token + "_" + tokens[t + 1] in self.verbs
                    ):
                        joined = False

                elif token in suffixes and result[-1] in self.words:
                    joined = True

            if joined:
                result.pop()
                result.append(token_pair)
            else:
                result.append(token)

        return result


    def seperate_mi(self,text):

        matches = re.findall(r"\bن?می[آابپتثجچحخدذرزژسشصضطظعغفقکگلمنوهی]+", text)
        for m in matches:
            r = re.sub("^(ن?می)", r"\1‌", m)
            if r in self.verbs:
                text = text.replace(m, r)

        return text


    def normalize(self,text):
        text = self.normalize_alphabets(text)
        text = self.normalize_unicodes(text)
        text = self.normalize_special_characters(text)
        text = self.correct_spacing( text)
        text = self.seperate_mi(text)
        return text


In [5]:
text = documents[0]
my_normalizer = MyNormalizer()
print(my_normalizer.normalize(text))


به گزارش خبرگزاری فارس کنفدراسیون فوتبال آسیا AFC در نامه‌ای رسمی به فدراسیون فوتبال ایران و باشگاه گیتی پسند زمان  قرعه‌کشی جام باشگاه‌های فوتسال آسیا را رسما اعلام کرد بر این اساس ۲۵ فروردین‌ماه ۱۴۰۱ مراسم قرعه‌کشی جام باشگاه‌های فوتسال آسیا در مالزی برگزار می‌شود باشگاه گیتی پسند بعنوان قهرمان فوتسال ایران در سال ۱۴۰۰ به این مسابقات راه پیدا کرده است پیش از این گیتی پسند تجربه ۳ دوره حضور در جام باشگاه‌های فوتسال آسیا را داشته که هر سه دوره به فینال مسابقات راه پیدا کرده و یک عنوان قهرمانی و دو مقام دومی بدست آورده است انتهای پیام



## Defining tokenizer

In [6]:
def tokenize_text(text,my_normalizer):
    # Regular expression patterns for identifying emails and numbers
    # email_pattern = re.compile(r"[a-zA-Z0-9._+-]+@([a-zA-Z0-9-]+\.)+[A-Za-z]{2,}",)
    # number_int_pattern = re.compile(r"\b(?<![\d۰-۹][.٫٬,])([\d۰-۹]+)(?![.٫٬,][\d۰-۹])\b",)
    # number_float_pattern = re.compile(r"\b(?<!\.)([\d۰-۹,٬]+[.٫٬][\d۰-۹]+)\b(?!\.)",)
    # id_pattern = re.compile(r"(?<![\w._])(@[\w_]+)")

    # Combine the email and number patterns into one pattern
    # combined_pattern = f'({email_pattern})|({number_int_pattern})|({number_float_pattern})|({id_pattern})'

    tokens = re.split(r'\t|\n| |‌', text)

    tokens = [token for token in tokens if token]
    
    text = " ".join(tokens)

    text = my_normalizer.correct_spacing(text)

    words = re.split(r' ', text)

    # Tokenize the words using the combined pattern
    # tokens = [re.search(combined_pattern, word).group() if re.search(combined_pattern, word) else word for word in words]

    return words

# Example usage
text_to_tokenize = "من فاطمه عبدی با شماره دانشجویی9931089 و ایمیل fatemehabdina@gmail.com و آیدی @fatem_abdi هستم"
tokens = tokenize_text(text_to_tokenize, my_normalizer)
print(tokens)

['من', 'فاطمه', 'عبدی', 'با', 'شماره', 'دانشجویی', '9931089', 'و', 'ایمیل', 'fatemehabdina@gmail.com', 'و', 'آیدی', '@fatem_abdi', 'هستم']


## Preprocessing text

### Normalizing, tokenizing, stemming

In [7]:

stemmer = FindStems()


preprocessed_tokens = []

for content in documents:

    normalized_content = my_normalizer.normalize(content)

    content_tokens = tokenize_text(normalized_content,my_normalizer=my_normalizer)

    stemmed_tokens = [stemmer.convert_to_stem(token) for token in content_tokens]

    preprocessed_tokens.append(stemmed_tokens)

### Removing stopwords

In [8]:

word_frequencies = {}
for tokens in preprocessed_tokens:
    for token in tokens:
        word_frequencies[token] = word_frequencies.get(token, 0) + 1

# Sort words based on their frequencies
sorted_words = sorted(word_frequencies.items(), key=lambda x: x[1], reverse=True)

# Select the top 50 most frequent words to remove
stop_words = [word for word, frequency in sorted_words[:50]]

In [9]:
# Remove the selected words from the documents
final_tokens = [ [token for token in tokens if token not in stop_words] for tokens in preprocessed_tokens]


In [10]:
print("List of stopwords and their frequencies:")
for i , word in enumerate(stop_words):
    print(f"{i+1}- {word}: {word_frequencies[word]}")

List of stopwords and their frequencies:
1- و: 218109
2- در: 163370
3- به: 132417
4- از: 92525
5- این: 81940
6- که: 75067
7- با: 68524
8- را: 67199
9- اس: 56763
10- کرد&کن: 44429
11- برای: 30913
12- داشت&دار: 30598
13- شد&شو: 28741
14- تیم: 27477
15- کرد: 24111
16- بود&باش: 23918
17- هم: 21656
18- کشور: 21544
19- ما: 19612
20- شد: 18970
21- بازی: 17692
22- یک: 17559
23- آن: 16667
24- باید: 16107
25- تا: 15808
26- بر: 15622
27- وی: 15345
28- داد&ده: 14620
29- خود: 14494
30- خواست&خواه: 14380
31- مجلس: 14290
32- اسلامی: 14241
33- گزارش: 13909
34- گفت: 13303
35- فارس: 13245
36- مردم: 13046
37- ایران: 12501
38- خبرگزاری: 12399
39- پیام: 12252
40- سال: 12212
41- دولت: 12165
42- انتهای: 12104
43- اما: 12026
44- گرفت&گیر: 11376
45- توانست&توان: 11063
46- بازیکن: 10980
47- داشت&دارد: 10568
48- اینکه: 10106
49- ملی: 9762
50- کار: 9638


# Positional indexing

In [11]:
positional_index = defaultdict(lambda: {"total_count": 0, "postings_lists": defaultdict(lambda: {"count": 0, "positions": [],"tf_idf":0})})

for doc_id, tokens in enumerate(final_tokens):
    for position, token in enumerate(tokens):

        positional_index[token]["total_count"] += 1

        positional_index[token]["postings_lists"][doc_id]["count"] += 1
        positional_index[token]["postings_lists"][doc_id]["positions"].append(position)

In [12]:
# Example of how to access information in the positional index

example_token = "امیرکبیر"
total_count = positional_index[example_token]["total_count"]
document_frequency = positional_index[example_token]["postings_lists"]

print(f"Token '{example_token}' appears {total_count} times in the entire document collection.")
print(f"Document frequency of '{example_token}': {len(document_frequency)} documents.")
for doc_id, info in document_frequency.items():
    print(f" - In document {doc_id}, it appears {info['count']} times at positions {info['positions']}")

Token 'امیرکبیر' appears 43 times in the entire document collection.
Document frequency of 'امیرکبیر': 24 documents.
 - In document 7346, it appears 2 times at positions [7, 98]
 - In document 7395, it appears 1 times at positions [14]
 - In document 7416, it appears 1 times at positions [21]
 - In document 7579, it appears 1 times at positions [955]
 - In document 7634, it appears 1 times at positions [1888]
 - In document 8125, it appears 4 times at positions [19, 31, 59, 122]
 - In document 8463, it appears 2 times at positions [1032, 1068]
 - In document 8645, it appears 1 times at positions [16]
 - In document 9261, it appears 1 times at positions [38]
 - In document 9286, it appears 4 times at positions [852, 1027, 1063, 1285]
 - In document 9808, it appears 1 times at positions [70]
 - In document 9845, it appears 1 times at positions [1303]
 - In document 10024, it appears 2 times at positions [92, 115]
 - In document 10361, it appears 1 times at positions [67]
 - In document 1

# Vectorizing documents

In [13]:
collection_size = len(final_tokens)
print(collection_size)

12202


In [14]:
# Update the positional index with TF-IDF values
for term, docs_info in positional_index.items():
    for doc_id, _ in docs_info["postings_lists"].items():
        tf_value = positional_index[term]["postings_lists"][doc_id]["count"]
        idf_value = len(positional_index[term]["postings_lists"])
        tfidf_value = (1 + math.log10(tf_value)) * math.log10(collection_size/idf_value)
        positional_index[term]["postings_lists"][doc_id]["tf_idf"] = tfidf_value

In [15]:
print(positional_index['امیرکبیر']["postings_lists"][7346]["tf_idf"])

3.520873107266285


# Create Champion lists

In [24]:
def create_champion_list(term, k):

    postings_list = positional_index[term]["postings_lists"]

    sorted_postings = sorted(postings_list.items(), key=lambda x: x[1]["tf_idf"]/len(final_tokens[x[0]]), reverse=True)

    champion_list = sorted_postings[:k]

    return champion_list

def create_all_champion_lists(k):
    champion_lists = {}

    for term in positional_index.keys():
        champion_lists[term] = create_champion_list(term, k)

    return champion_lists

champion_lists = create_all_champion_lists(150)

In [25]:
print(champion_lists['امیرکبیر'])

[(10361, {'count': 1, 'positions': [67], 'tf_idf': 2.706219778944763}), (9808, {'count': 1, 'positions': [70], 'tf_idf': 2.706219778944763}), (8125, {'count': 4, 'positions': [19, 31, 59, 122], 'tf_idf': 4.335526435587807}), (11603, {'count': 6, 'positions': [14, 22, 50, 98, 171, 235], 'tf_idf': 4.812068083743577}), (10567, {'count': 3, 'positions': [153, 184, 203], 'tf_idf': 3.9974147554220556}), (10024, {'count': 2, 'positions': [92, 115], 'tf_idf': 3.520873107266285}), (9261, {'count': 1, 'positions': [38], 'tf_idf': 2.706219778944763}), (8645, {'count': 1, 'positions': [16], 'tf_idf': 2.706219778944763}), (11645, {'count': 1, 'positions': [16], 'tf_idf': 2.706219778944763}), (7346, {'count': 2, 'positions': [7, 98], 'tf_idf': 3.520873107266285}), (12134, {'count': 1, 'positions': [371], 'tf_idf': 2.706219778944763}), (7395, {'count': 1, 'positions': [14], 'tf_idf': 2.706219778944763}), (7416, {'count': 1, 'positions': [21], 'tf_idf': 2.706219778944763}), (10681, {'count': 1, 'posit

# Query Answering

## Query preprocessing

In [26]:
def query_preprocess(query):
    normalized_query = my_normalizer.normalize(query)
    query_tokens = tokenize_text(normalized_query,my_normalizer)
    stemmed_query = [stemmer.convert_to_stem(token) for token in query_tokens]
    final_query = [token for token in stemmed_query if token not in stop_words]
    return final_query

## Check for query validation

In [27]:
def is_query_valid(preprocessed_query):
    if len(preprocessed_query) == 0:
        return False
    return True

## Calculate tf for query

In [28]:
def calculate_tf_query(preprocessed_query):
    tf = {}
    for term in preprocessed_query:
        tf[term] = tf.get(term, 0) + 1
    for term,frequency in tf.items():
        # print(term, tf[term])
        tf[term] =(1 + math.log(tf[term]))
        # print(term, tf[term])

    return tf

## Cosine similarity

In [29]:
def cosine_similarity(query_with_tf,tokens,use_champion_list):
    multiplied_tfidf_docs = defaultdict(lambda: 0)
    if(use_champion_list == False):
        for term,tf_term in query_with_tf.items():
            if(term not in champion_lists.keys()):
                continue
            for doc_id, term_doc_info in positional_index[term]["postings_lists"].items():
                mul_tf = tf_term * positional_index[term]["postings_lists"][doc_id]["tf_idf"]
                multiplied_tfidf_docs[doc_id] = multiplied_tfidf_docs[doc_id] + mul_tf
    else:
        for term,tf_term in query_with_tf.items():
            if(term not in champion_lists.keys()):
                continue
            for doc in champion_lists[term]:
                doc_id = doc[0]
                tf_idf_doc = doc[1]["tf_idf"]
                mul_tf = tf_term * tf_idf_doc
                multiplied_tfidf_docs[doc_id] = multiplied_tfidf_docs[doc_id] + mul_tf


    for doc_id, mul_tf in multiplied_tfidf_docs.items():
        len_doc = len(tokens[doc_id])
        multiplied_tfidf_docs[doc_id] /= (len_doc*len(query_with_tf))

    return multiplied_tfidf_docs

## Ranked retrieval

In [30]:
def top_k_docs(query_tf,k):
    cosine_similarities =  cosine_similarity(query_tf,final_tokens,True)
    if(len(cosine_similarities) < k):
        cosine_similarities =  cosine_similarity(query_tf,final_tokens,False)
        sorted_cosine_similarities = sorted(cosine_similarities.items(), key=lambda item: item[1], reverse=True)
        return sorted_cosine_similarities
    else:
        sorted_cosine_similarities = sorted(cosine_similarities.items(), key=lambda item: item[1], reverse=True)
        return sorted_cosine_similarities[:k]


In [38]:
urls =[data[i]['url'] for i in data]
titles = [data[i]['title'] for i in data]
def show_results(query,top_answers):

    print(f"Query: {query}\nDocuments:\n")

    for doc_id, cosine_sim in top_answers:
        print(f"  Document ID: {doc_id}")
        print(f"    URL: {urls[doc_id]}")
        print(f"    Title: {titles[doc_id]}")
        print(f"    Cosine similarity: {cosine_sim}")



In [32]:
def execute_search_engine(query, number_of_retrieved_docs):
    preprocessed_query = query_preprocess(query)
    query_is_valid = is_query_valid(preprocessed_query)
    if(query_is_valid == False):
        print("Need more details")
        return
    query_tf = calculate_tf_query(preprocessed_query)
    top_answers = top_k_docs(query_tf,number_of_retrieved_docs)

    show_results(query,top_answers)


# Report

## Part 1

In [33]:
query = 'ایران'
k = 5
execute_search_engine(query,k)

Need more details


In [34]:
query = 'فارس'
k = 5
execute_search_engine(query,k)

Need more details


In [39]:
query = 'اخبار'
k = 5
execute_search_engine(query,k)

Query: اخبار
Documents:

  Document ID: 11168
    URL: https://www.farsnews.ir/news/14000820000673/مدیر-اخبار-دفتر-رئیس-جمهور-منصوب-شد
    Title: مدیر اخبار دفتر رئیس جمهور منصوب شد
    Cosine similarity: 0.06896630021727439
  Document ID: 3439
    URL: https://www.farsnews.ir/news/14001108000590/کاپیتان-تراکتور-به-تیم-ملی-دعوت-شد
    Title: کاپیتان تراکتور به تیم ملی دعوت شد
    Cosine similarity: 0.04486361263632067
  Document ID: 7027
    URL: https://www.farsnews.ir/news/14001219000246/اعضای-مجلس-خبرگان-با-رهبر-انقلاب-دیدار-کردند
    Title: اعضای مجلس خبرگان با رهبر انقلاب دیدار کردند
    Cosine similarity: 0.04388764559281098
  Document ID: 8453
    URL: https://www.farsnews.ir/news/14001104000352/رئیس‌جمهور-به‌-صورت-زنده-با-مردم-صحبت-می‌کند
    Title: رئیس‌جمهور به‌ صورت زنده با مردم صحبت می‌کند
    Cosine similarity: 0.04388764559281098
  Document ID: 8233
    URL: https://www.farsnews.ir/news/14001110000183/جمعی-از-تولیدکنندگان-با-رهبر-انقلاب-دیدار-کردند
    Title: جمعی از تولی

## Part 2

In [45]:
query = 'استقلال و پرسپولیس'
k = 5
execute_search_engine(query,k)

Query: استقلال و پرسپولیس
Documents:

  Document ID: 2754
    URL: https://www.farsnews.ir/news/14001117000919/بازیکن-سابق-پرسپولیس-و-استقلال-به-پدیده-پیوست
    Title: بازیکن سابق پرسپولیس و استقلال به پدیده پیوست
    Cosine similarity: 0.04365571007008382
  Document ID: 5437
    URL: https://www.farsnews.ir/news/14001014000922/برتری-پرسپولیس-مقابل-استقلال-دربی-امیدها
    Title: برتری پرسپولیس مقابل استقلال دربی امیدها
    Cosine similarity: 0.03379732302454984
  Document ID: 198
    URL: https://www.farsnews.ir/news/14001221000810/تغییر-در-ساعت-دیدار-پرسپولیس-و-استقلال
    Title: تغییر در ساعت دیدار پرسپولیس و استقلال
    Cosine similarity: 0.03307584554723747
  Document ID: 1048
    URL: https://www.farsnews.ir/news/14001209001094/مدافع-پرسپولیس-بازی-مقابل-صنعت-نفت-را-از-دست-داد
    Title: مدافع پرسپولیس بازی مقابل صنعت نفت را از دست داد
    Cosine similarity: 0.03155935717957591
  Document ID: 1977
    URL: https://www.farsnews.ir/news/14001128000707/تساوی-پرگل-استقلال-و-پرسپولیس-در

## Part 3

In [46]:
query = 'کمیسیون'
k = 5
execute_search_engine(query,k)

Query: کمیسیون
Documents:

  Document ID: 9807
    URL: https://www.farsnews.ir/news/14000923000737/نوروزی-دلخوش-و-موحد-عضو-کمیسیون-تلفیق-بودجه-۱۴۰۱-شدند
    Title: نوروزی، دلخوش و موحد عضو کمیسیون تلفیق بودجه ۱۴۰۱ شدند
    Cosine similarity: 0.04559174736147669
  Document ID: 10084
    URL: https://www.farsnews.ir/news/14000916000703/اعضای-کمیسیون-فرهنگی-به-طبس-و-رفسنجان-سفر-می‌کنند
    Title: اعضای کمیسیون فرهنگی به طبس و رفسنجان سفر می‌کنند
    Cosine similarity: 0.045096027601516965
  Document ID: 9428
    URL: https://www.farsnews.ir/news/14001005000638/قاضی‌زاده‌-طباطبایی‌نژاد-و-یزدی‌خواه-تلفیقی‌های-کمیسیون-فرهنگی-شدند
    Title: قاضی‌زاده‌، طباطبایی‌نژاد و یزدی‌خواه تلفیقی‌های کمیسیون فرهنگی شدند
    Cosine similarity: 0.041647699416419354
  Document ID: 8274
    URL: https://www.farsnews.ir/news/14001108000634/دستور-جلسات-هفته-آینده-کمیسیون‌های-مجلس
    Title: دستور جلسات هفته آینده کمیسیون‌های مجلس
    Cosine similarity: 0.039638450882342385
  Document ID: 9084
    URL: https:

## Part 4

In [50]:
query = 'خودایمنی مورفه‌آ'
k = 5
execute_search_engine(query,k)

Query: خودایمنی مورفه‌آ
Documents:



In [58]:
query = 'مرض فلج اطفال'
k = 50
execute_search_engine(query,k)


Query: مرض فلج اطفال
Documents:

  Document ID: 10964
    URL: https://www.farsnews.ir/news/14000825000739/3-سازمان-متولی-موضوع-کودکان-کار-هستند-اما-آماری-از-تعداد-آنها-وجود
    Title: 13 سازمان متولی موضوع کودکان کار هستند اما آماری از تعداد آنها وجود ندارد
    Cosine similarity: 0.010558700088873958
  Document ID: 7805
    URL: https://www.farsnews.ir/news/14001120000688/انقلاب-اسلامی-اقتصاد-مقاومتی-تولید-ملی-کارآفرینی-علت-ناکامی‌ها-در
    Title: انقلاب اسلامی، اقتصاد مقاومتی، تولید ملی، کارآفرینی/ علت ناکامی‌ها در مدیریت اقتصاد کشور، مدیران غربگرا بود
    Cosine similarity: 0.007433801753321521
  Document ID: 9385
    URL: https://www.farsnews.ir/news/14001007000122/برای-کاهش-خام‌فروشی-در-دولت-قبل-اقدامی-نشد-بی‌توجهی-وزیر-سابق-نفت-به
    Title: برای کاهش خام‌فروشی در دولت قبل اقدامی نشد/ بی‌توجهی وزیر سابق نفت به دور زدن تحریم‌ها
    Cosine similarity: 0.003855803928056789
  Document ID: 9386
    URL: https://www.farsnews.ir/news/14001007000097/محورهای-تحقیق-و-تفحص-از-عملکرد-وزارت-ن

# Exam

In [153]:
#Question 1

text = documents[4092]
tokenized_text = tokenize_text(text,my_normalizer)
tokenized_text_for_normalizing = " ".join(tokenized_text)
normalized_text = my_normalizer.normalize(tokenized_text_for_normalizing)
normalized_and_tokenized_text = tokenize_text(normalized_text,my_normalizer)


print(text)
print(tokenized_text)
print(normalized_and_tokenized_text)
changed_tokens = []
for i,token in enumerate(normalized_and_tokenized_text):
    if(token not in tokenized_text):
        changed_tokens.append(token)

print(changed_tokens)
print(len(changed_tokens))
comparing_list = []
for token in tokenized_text:
    normalized_token = my_normalizer.normalize(token)
    if(normalized_token != token):
        comparing_list.append((token,normalized_token))

print(comparing_list)
print(len(comparing_list))


به گزارش خبرنگار پارلمانی خبرگزاری فارس، سید مرتضی حسینی نماینده مردم میانه در نطق میان دستور خود در نشست علنی امروز (یکشنبه 21 آذر ماه) مجلس شورای اسلامی گفت: در شعارهای رئیس جمهور محترم مسأله مردمی بودن و از مردم شنیدن و در میان مردم بودن تکرار شده است، این را از دست ندهند. نماینده مردم میانه در مجلس شورای اسلامی اظهار داشت: اگر ما به معنای واقعی کلمه می خواهیم در کنار مردم باشیم، باید بی‌امان با فساد و مفسد مبارزه کنیم. بسترها و زمینه‌های فساد در قوه مجریه تشکیل می‌شود. اینجا باید با فساد مبارزه کرد. وی افزود: فرارهای مالیاتی، انحصار‌های بی‌بدیل، سوء استفاده از ارز ترجیحی، فعالیت‌های ناسالم سوداگرایانه، قاچاق و امثال اینها فسادهایی است که باید با پیگیری و با برنامه و با همت ادامه پیدا کند تا به نتیجه برسد. حسینی خاطرنشان کرد: کمبودها و مشکلات زیاد است، به ویژه مشکل مسکن، مشکل بیکاری جوانان، مشکل تورم و گرانی، مشکل قاچاق کالا و واردات بی‌رویه، اما به قول حضرت آقا ظرفیت‌ها زیادتر از مشکلات است. نماینده مردم میانه در مجلس شورای اسلامی اظهار داشت: ظرفیت‌های فراوانی در کشور وجود دارد. م

In [ ]:
filtered_text = []
for token in normalized_and_tokenized_text:
    if(token not in stop_words):
        filtered_text.append(token)

deleted_words = []
for token in normalized_and_tokenized_text:
    if(token not in filtered_text):
        deleted_words.append(token)
print(deleted_words)

In [ ]:
verbs = my_normalizer.verbs
stemmed_tokens_4092 = [(stemmer.convert_to_stem(token),token) for token in filtered_text]
stemming_impact = []
for stemmed_token,token in stemmed_tokens_4092:
    if(stemmed_token!= token):
        if(token in verbs):
            stemming_impact.append((token,stemmed_token))
print(stemming_impact)

In [ ]:
import sys
print(sys.getsizeof(positional_index))

In [ ]:
sorted_postings_list_length = sorted(positional_index.items(), key=lambda x: len(x[1]["postings_lists"]), reverse=True)
print(sorted_postings_list_length[0][0],sorted_postings_list_length[1][0],sorted_postings_list_length[3][0])
print(sorted_postings_list_length[len(sorted_postings_list_length)-1][0],sorted_postings_list_length[len(sorted_postings_list_length)-1][1]["postings_lists"])
print(sorted_postings_list_length[len(sorted_postings_list_length)-2][0],sorted_postings_list_length[len(sorted_postings_list_length)-2][1]["postings_lists"])
print(sorted_postings_list_length[len(sorted_postings_list_length)-3][0],sorted_postings_list_length[len(sorted_postings_list_length)-3][1]["postings_lists"])


In [ ]:

dictionary_size = len(positional_index)
print(dictionary_size)

In [ ]:
# Update the positional index with TF-IDF values
min_idf = 9999999999999999999999999999999999
max_idf = -55555555555555555555555555555555
min_word = []
max_word = []
for term, docs_info in positional_index.items():
    for doc_id, _ in docs_info["postings_lists"].items():
        idf_value = len(positional_index[term]["postings_lists"])
        idf = math.log(collection_size/idf_value)
        if(idf>max_idf):
            max_idf = idf
            max_word = term
        if(idf<min_idf):
            min_idf = idf
            min_word = term

print(f"max idf: {max_idf} in term: {max_word}")
print(f"min idf: {min_idf} in term: {min_word}")

In [ ]:
# Update the positional index with TF-IDF values
min_tfidf = 9999999999999999999999999999999999
max_tfidf = -55555555555555555555555555555555
min_word = []
max_word = []
for term in stemmed_tokens:
    tfidf = positional_index[term]["postings_lists"][4092]["tf_idf"]
    if(tfidf>max_tfidf):
        max_tfidf = tfidf
        max_word = term
    if(tfidf<min_tfidf):
        min_tfidf = tfidf
        min_word = term 
    

print(f"max tfidf: {max_tfidf} in term: {max_word}")
print(f"min tfidf: {min_tfidf} in term: {min_word}")


Query:  1400 خبرنگار نمونه سال

Documents:

  Document ID: 1546
    URL: https://www.farsnews.ir/news/14000928000049/لیست-ممنوعه-سال-2022-از-یازدهم-دی-ماه-اجرایی-می-شود
    Title: لیست ممنوعه سال 2022 از یازدهم دی ماه اجرایی می شود
    Cosine similarity: 0.7903098637920106


  Document ID: 139
    URL: https://www.farsnews.ir/news/14001125000182/نگاهی-به-دوپینگ-اسکی‌باز-ایران-مشکلِ-عمدی-یا-دست‌های-پشت-پرده
    Title: نگاهی به دوپینگ اسکی‌باز ایران؛ مشکلِ عمدی یا دست‌های پشت پرده؟
    Cosine similarity: 0.7137726749655947


  Document ID: 4476
    URL: https://www.farsnews.ir/news/14001008000978/بُرشی-از-کتاب-فتنه-تغلب-روایت‌هایی-از-متن-و-فرامتن-آشوب‌های-‌۸۸
    Title: بُرشی از کتاب « فتنه تغلب»/ روایت‌هایی از متن و فرامتن آشوب‌های ‌۸۸
    Cosine similarity: 0.6764230826423696


  Document ID: 4517
    URL: https://www.farsnews.ir/news/14001004000648/دیوان-محاسبات-کسری-تراز-لایحه-بودجه-سال-آینده-35درصد-از-بودجه-امسال
    Title: دیوان محاسبات: کسری تراز لایحه بودجه سال آینده 35درصد از بو

In [ ]:
def top_k_docs3(query_tf,k,flag):
    cosine_similarities =  cosine_similarity(query_tf,final_tokens,flag)
    sorted_cosine_similarities = sorted(cosine_similarities.items(), key=lambda item: item[1], reverse=True)
    return sorted_cosine_similarities[:k]

In [ ]:
def execute_search_engine3(query, number_of_retrieved_docs,flag):
    preprocessed_query = query_preprocess(query)
    query_is_valid = is_query_valid(preprocessed_query)
    if(query_is_valid == False):
        print("Need more details")
        return
    query_tf = calculate_tf_query(preprocessed_query)
    top_answers = top_k_docs3(query_tf,number_of_retrieved_docs,flag)

    show_results(query,top_answers)

In [ ]:
def query_preprocess2(query):
    query_tokens = tokenize_text(query,my_normalizer)
    stemmed_query = [stemmer.convert_to_stem(token) for token in query_tokens]
    final_query = [token for token in stemmed_query if token not in stop_words]
    return final_query

In [ ]:
def execute_search_engine2(query, number_of_retrieved_docs):
    preprocessed_query = query_preprocess2(query)
    query_is_valid = is_query_valid(preprocessed_query)
    if(query_is_valid == False):
        print("Need more details")
        return
    query_tf = calculate_tf_query(preprocessed_query)
    top_answers = top_k_docs(query_tf,number_of_retrieved_docs)

    show_results(query,top_answers)



In [ ]:
query = ".میرود"
print("with normalizing")
execute_search_engine(query,10)

print("without normalizing")
execute_search_engine2(query,10)

In [ ]:
query = " 1400 خبرنگار نمونه سال"
k = 10
time_s_ch = time.time()
execute_search_engine3(query,k,True)
time_e_ch = time.time()

time_s_nch = time.time()
execute_search_engine3(query,k,False)
time_e_nch = time.time()

print("with champion list time (micro second):")
print((time_e_ch-time_s_ch)*1000000)
print("without champion list time (micro second):")
print((time_e_nch-time_s_nch)*1000000)



In [ ]:
fd = query_preprocess(first_doc)
sd = query_preprocess(documents[4476])
common_terms = []
for term in fd:
    if term in sd:
        common_terms.append(term)

print(len(common_terms))
print(len(fd),len(sd))